## Problem 1 — Hash Join Implementation

In [1]:
def hash_join_problem1(R1, R2):
    # Build a hash table on R2 using attribute B as the key.
    # For each value of B, we store all full tuples from R2 that have that B.
    h = {}
    for t in R2:                
        b = t[0]
        if b not in h:
            h[b] = []
        h[b].append(t)         

    # Now we scan R1 and look up matches in the hash table.
    result = []
    for (a, b) in R1:
        if b in h:
            # For each matching R2 tuple, produce the joined (A, B, C)
            for (_, c) in h[b]:
                result.append((a, b, c))
    return result

# Example dataset: 10 tuples in R1 and 10 in R2 
R1 = [(1,12),(2,5),(3,10),(4,17),(5,8),(6,19),(7,16),(8,8),(9,20),(10,19)]
R2 = [(8,100),(7,101),(19,102),(3,103),(1,104),(19,105),(11,106),(11,107),(20,108),(20,109)]

# Run the join
print("Output of Hash Join (A, B, C tuples):")
output = hash_join_problem1(R1, R2)
output

Output of Hash Join (A, B, C tuples):


[(5, 8, 100),
 (6, 19, 102),
 (6, 19, 105),
 (8, 8, 100),
 (9, 20, 108),
 (9, 20, 109),
 (10, 19, 102),
 (10, 19, 105)]

## Problem 2 — Yannakakis-Style Line Join

In [2]:
# q(A1, A2, ..., Ak+1) :- R1(A1,A2), R2(A2,A3), ..., Rk(Ak, Ak+1)

from collections import defaultdict

def semijoin_reduce(left, right, join_attr_left, join_attr_right):
   
    right_keys = {t[join_attr_right] for t in right}
    return [t for t in left if t[join_attr_left] in right_keys]

def yannakakis_line_join(relations):
    """
    Input:
        relations = [R1, R2, ..., Rk]
        where Ri is a list of tuples (Ai, Ai+1)

    Output:
        The full join result of the k-line path.
    """
    k = len(relations)

    # 1. Bottom-up pass (semijoins)

    # Start from Rk and move backwards to R1
    reduced = relations.copy()

    for i in range(k - 1, 0, -1):
        # Join attribute:
        # Ri joins with Ri+1 on Ai+1
        reduced[i-1] = semijoin_reduce(
            left=reduced[i-1],
            right=reduced[i],
            join_attr_left=1,   # Ai+1 in Ri
            join_attr_right=0   # Ai+1 in Ri+1
        )
   
    # 2. Top-down pass (semijoins)
   
    for i in range(k - 1):
        reduced[i+1] = semijoin_reduce(
            left=reduced[i+1],
            right=reduced[i],
            join_attr_left=0,   # Ai+1 in Ri+1
            join_attr_right=1   # Ai+1 in Ri
        )

    # 3. Final join (now all small)
    
    # Start building the result from R1
    result = [(a1, a2) for (a1, a2) in reduced[0]]  # (A1,A2)

    # Extend step-by-step through remaining relations
    for i in range(1, k):
        next_rel = reduced[i]
        
        # Build an index for fast lookups
        index = defaultdict(list)
        for (x, y) in next_rel:
            index[x].append(y)

        new_result = []
        for tup in result:
            last_val = tup[-1]  # the Ai+1 value
            if last_val in index:
                for nxt in index[last_val]:
                    new_result.append(tup + (nxt,))
        result = new_result

    return result

# Example: 3-line join (R1, R2, R3)

R1 = [(1,2), (2,3), (3,4), (4,5)]
R2 = [(2,10), (3,11), (4,12), (5,13)]
R3 = [(10,100), (11,101), (12,102), (13,103)]

relations = [R1, R2, R3]

print("Output of Yannakakis Join (A1, A2, ..., Ak+1 tuples):")
output = yannakakis_line_join(relations)
output

Output of Yannakakis Join (A1, A2, ..., Ak+1 tuples):


[(1, 2, 10, 100), (2, 3, 11, 101), (3, 4, 12, 102), (4, 5, 13, 103)]

## Problem 3 — Naive Cascading Line Join

In [3]:
# q(A1, ..., Ak+1) :- R1(A1,A2), R2(A2,A3), ..., Rk(Ak,Ak+1)

from collections import defaultdict

# We reuse the hash join idea from Problem 1.
def hash_join_two(rel_left, rel_right):
    """
    Join two binary relations:
    rel_left:  list of tuples (..., x)
    rel_right: list of tuples (x, y)
    Output: joined tuples (..., x, y)
    """
    # Build a hash table on the first attribute of rel_right.
    h = defaultdict(list)
    for (x, y) in rel_right:
        h[x].append(y)

    result = []
    for t in rel_left:
        x = t[-1]  # the last attribute is the join key
        if x in h:
            for y in h[x]:
                result.append(t + (y,))
    return result

def naive_line_join(relations):
    """
    Input:
        relations = [R1, R2, ..., Rk]
        where Ri is a list of tuples (Ai, Ai+1)

    Output:
        All joined tuples from R1 ⋈ R2 ⋈ ... ⋈ Rk
    """
    # Start by converting R1(A1,A2) into tuples (A1,A2)
    result = [(a1, a2) for (a1, a2) in relations[0]]

    # Iteratively join with R2, R3, ..., Rk
    for i in range(1, len(relations)):
        result = hash_join_two(result, relations[i])
    return result

# Example dataset for a 3-line join

R1 = [(1,2), (2,3), (3,4), (4,5)]
R2 = [(2,10), (3,11), (4,12), (5,13)]
R3 = [(10,100), (11,101), (12,102), (13,103)]

relations = [R1, R2, R3]

print("Output — Naive Line Join (A1, A2, ..., Ak+1 tuples):")
output_problem3 = naive_line_join(relations)
output_problem3

Output — Naive Line Join (A1, A2, ..., Ak+1 tuples):


[(1, 2, 10, 100), (2, 3, 11, 101), (3, 4, 12, 102), (4, 5, 13, 103)]

## Problem 4 — Random Dataset for 3-Line Join

In [4]:
import random
import time

# Create the 3 relations 
# R1: (i, x) for i=1..100, x random in [1,5000]
R1_100 = [(i, random.randint(1, 5000)) for i in range(1, 101)]

# R2: (y, j) for j=1..100, y random in [1,5000]
R2_100 = [(random.randint(1, 5000), j) for j in range(1, 101)]

# R3: (ℓ, ℓ) for ℓ=1..100
R3_100 = [(l, l) for l in range(1, 101)]

relations_3line = [R1_100, R2_100, R3_100]

#Run Problem 2
start_p2 = time.perf_counter()
output_p2 = yannakakis_line_join(relations_3line)
time_p2 = time.perf_counter() - start_p2

#Run Problem 3
start_p3 = time.perf_counter()
output_p3 = naive_line_join(relations_3line)
time_p3 = time.perf_counter() - start_p3

same_output = set(output_p2) == set(output_p3)
len_output = len(output_p2)

print("Yannakakis Time: ",time_p2)
print("Naive Time:  ", time_p3)
print("Same Output? ",same_output)
print("Output Size: ",len_output )

Yannakakis Time:  0.000372900627553463
Naive Time:   0.00045559927821159363
Same Output?  True
Output Size:  4


## Problem 5 — Large Structured Dataset for 3-Line Join

In [5]:

import random
import time
import pandas as pd
import os

# PROBLEM 5 — Dataset Construction 

# Construct R1
R1_p5 = []
R1_p5 += [(i, 5) for i in range(1, 1001)]
R1_p5 += [(i, 7) for i in range(1001, 2001)]
R1_p5.append((2001, 2002))
random.shuffle(R1_p5)

# Construct R2
R2_p5 = []
R2_p5 += [(5, i) for i in range(1, 1001)]
R2_p5 += [(7, i) for i in range(1001, 2001)]
R2_p5.append((2002, 8))
random.shuffle(R2_p5)

# Construct R3
R3_p5 = [(random.randint(2002,3000), random.randint(1,3000)) for _ in range(2000)]
R3_p5.append((8, 30))
random.shuffle(R3_p5)

relations_p5 = [R1_p5, R2_p5, R3_p5]

# Run Problem 2 implementation (Yannakakis line join)

t0 = time.perf_counter()
out_p2 = yannakakis_line_join(relations_p5)
t1 = time.perf_counter()
time_p2 = t1 - t0

# Run Problem 3 implementation (Naive hash-chain join)

t2 = time.perf_counter()
out_p3 = naive_line_join(relations_p5)
t3 = time.perf_counter()
time_p3 = t3 - t2

# CREATE FOLDER AND SAVE CSVs INSIDE IT

folder = "DatasetforThreeLineQuery"
os.makedirs(folder, exist_ok=True)

df_R1 = pd.DataFrame(R1_p5, columns=["A1", "A2"])
df_R2 = pd.DataFrame(R2_p5, columns=["A2", "A3"])
df_R3 = pd.DataFrame(R3_p5, columns=["A3", "A4"])

df_R1.to_csv(f"{folder}/R1_p5.csv", index=False)
df_R2.to_csv(f"{folder}/R2_p5.csv", index=False)
df_R3.to_csv(f"{folder}/R3_p5.csv", index=False)

print("Saved Problem 5 datasets in folder: DatasetforThreelineQuery/")
print("Files created: R1_p5.csv, R2_p5.csv, R3_p5.csv")

# PRINT METRICS

print("Output size (Problem 2):", len(out_p2))
print("Output size (Problem 3):", len(out_p3))
print("Same outputs?:", set(out_p2) == set(out_p3))
print("Problem 2 runtime:", time_p2)
print("Problem 3 runtime:", time_p3)

Saved Problem 5 datasets in folder: DatasetforThreelineQuery/
Files created: R1_p5.csv, R2_p5.csv, R3_p5.csv
Output size (Problem 2): 1001
Output size (Problem 3): 1001
Same outputs?: True
Problem 2 runtime: 0.002710900269448757
Problem 3 runtime: 0.9798926999792457


## Problem 6 is in the SQL file outside this IPYNB File named Problem6.sql

## Problem 7 — Generic Join, GHW, and FHW Implementations

In [6]:
import pandas as pd
import time
from collections import defaultdict

# LOAD ALL SEVEN RELATIONS

R1 = pd.read_csv("QueryRelations/R1.csv")
R2 = pd.read_csv("QueryRelations/R2.csv")
R3 = pd.read_csv("QueryRelations/R3.csv")
R4 = pd.read_csv("QueryRelations/R4.csv")
R5 = pd.read_csv("QueryRelations/R5.csv")
R6 = pd.read_csv("QueryRelations/R6.csv")
R7 = pd.read_csv("QueryRelations/R7.csv")

# GENERIC JOIN, GHW JOIN, FHW JOIN

# ---- Helper: DF → list of tuples ----
def df_to_tuples(df):
    return [tuple(x) for x in df.to_numpy()]

# ---- Helper: DF → list of dicts ----
def to_dict_list(df):
    return df.to_dict("records")

# ---- Make separate clean copies for Generic Join ----
R1_copy = R1.copy()
R2_copy = R2.copy()
R3_copy = R3.copy()
R4_copy = R4.copy()
R5_copy = R5.copy()
R6_copy = R6.copy()
R7_copy = R7.copy()

R1t = df_to_tuples(R1_copy)
R2t = df_to_tuples(R2_copy)
R3t = df_to_tuples(R3_copy)
R4t = df_to_tuples(R4_copy)
R5t = df_to_tuples(R5_copy)
R6t = df_to_tuples(R6_copy)
R7t = df_to_tuples(R7_copy)

# ATTRIBUTE SCHEMAS

A1 = ["A1","A2"]
A2 = ["A2","A3"]
A3 = ["A1","A3"]
A4 = ["A3","A4"]
A5 = ["A4","A5"]
A6 = ["A5","A6"]
A7 = ["A4","A6"]

# GENERIC JOIN (AGM)

def project(R, idx):
    return set(row[idx] for row in R)

def filter_eq(R, idx, value):
    return [row for row in R if row[idx] == value]

def generic_join(relations, attributes):
    if not attributes:
        return [dict()]

    X = attributes[0]

    R_with_X = [(R, A.index(X), A) for (R,A) in relations if X in A]

    domain = None
    for (R, idx, A) in R_with_X:
        vals = project(R, idx)
        domain = vals if domain is None else domain & vals

    outputs = []
    for val in domain:
        new_rel = []
        for (R, A) in relations:
            if X in A:
                idx = A.index(X)
                Rf = filter_eq(R, idx, val)
                new_rel.append((Rf, A))
            else:
                new_rel.append((R, A))

        subs = generic_join(new_rel, attributes[1:])
        for s in subs:
            s[X] = val
            outputs.append(s)

    return outputs

# GHW (YANNAKAKIS) JOIN

def semijoin(parent, child, pkey, ckey):
    S = set(t[pkey] for t in parent)
    return [t for t in child if t[ckey] in S]

def ghw_join(R1,R2,R3,R4,R5,R6,R7):

    # convert to dicts
    R1d = to_dict_list(R1)
    R2d = to_dict_list(R2)
    R3d = to_dict_list(R3)
    R4d = to_dict_list(R4)
    R5d = to_dict_list(R5)
    R6d = to_dict_list(R6)
    R7d = to_dict_list(R7)

    # bottom-up semijoins
    R6d = semijoin(R5d, R6d, "A5", "A5")
    R7d = semijoin(R4d, R7d, "A4", "A4")
    R5d = semijoin(R4d, R5d, "A4", "A4")
    R4d = semijoin(R3d, R4d, "A3", "A3")
    R3d = semijoin(R1d, R3d, "A1", "A1")
    R2d = semijoin(R1d, R2d, "A2", "A2")

    # top-down pruning
    R1d = semijoin(R3d, semijoin(R2d, R1d, "A2", "A2"), "A1", "A1")

    # final merge-join pipeline
    out = (
        pd.DataFrame(R1d)
          .merge(pd.DataFrame(R2d), on="A2")
          .merge(pd.DataFrame(R3d), on=["A1","A3"])
          .merge(pd.DataFrame(R4d), on="A3")
          .merge(pd.DataFrame(R5d), on="A4")
          .merge(pd.DataFrame(R6d), on="A5")
          .merge(pd.DataFrame(R7d), on=["A4","A6"])
    )
    return out

# FHW JOIN = same as GHW for this query

def fhw_join(R1,R2,R3,R4,R5,R6,R7):
    return ghw_join(R1,R2,R3,R4,R5,R6,R7)

# RUN ALL THREE ALGORITHMS

full_attrs = ["A1","A2","A3","A4","A5","A6"]

relations_set = [
    (R1t, A1), (R2t, A2), (R3t, A3),
    (R4t, A4), (R5t, A5), (R6t, A6), (R7t, A7)
]

# --- GENERIC JOIN ---
t0 = time.perf_counter()
gj_out = generic_join(relations_set, full_attrs)
generic_time = time.perf_counter() - t0
generic_size = len(gj_out)

# --- GHW ---
t0 = time.perf_counter()
ghw_out = ghw_join(R1.copy(), R2.copy(), R3.copy(),
                   R4.copy(), R5.copy(), R6.copy(), R7.copy())
ghw_time = time.perf_counter() - t0
ghw_size = len(ghw_out)

# --- FHW ---
t0 = time.perf_counter()
fhw_out = fhw_join(R1.copy(), R2.copy(), R3.copy(),
                   R4.copy(), R5.copy(), R6.copy(), R7.copy())
fhw_time = time.perf_counter() - t0
fhw_size = len(fhw_out)

print(f"Generic Join runtime:    {generic_time:.6f} seconds")
print(f"GHW Join runtime:        {ghw_time:.6f} seconds")
print(f"FHW Join runtime:        {fhw_time:.6f} seconds")
print()
print(f"Generic Join output size: {generic_size}")
print(f"GHW Join output size:     {ghw_size}")
print(f"FHW Join output size:     {fhw_size}")

Generic Join runtime:    9.875571 seconds
GHW Join runtime:        0.457814 seconds
FHW Join runtime:        0.381953 seconds

Generic Join output size: 1000000
GHW Join output size:     1000000
FHW Join output size:     1000000
